# WISE COMPUTING HEAVENS: 
## Project: Binary Neutron Star Merger GW 170817
Instructor: Ian Johnson

Date: March 24 2025

In [ ]:
import numpy as np 
import matplotlib.pyplot as plt
from scipy.signal import butter, filtfilt, spectrogram

In [ ]:
# Load data
# Find your full file path
data = np.loadtxt("H-H1_GWOSC_16KHZ_R1-1187008867-32.txt")

# Define time array
fs = 16384  # Sampling rate in Hz
duration = 32  # Seconds
t = np.linspace(0, duration, len(data), endpoint=False)


In [ ]:

# Cut the data around the merger 
start_time = 0
end_time = 32

# Find the indices corresponding to the time range
start_idx = int(start_time * fs)
end_idx = int(end_time * fs)

# Slice the data and time array to the desired range
data_cut = data[start_idx:end_idx]
t_cut = t[start_idx:end_idx]

# Plot the sliced data
plt.figure(figsize=(10, 4))
plt.plot(t_cut, data_cut, label="Cut Data", alpha=0.7)
plt.xlabel("Time (s)")
plt.ylabel("Strain")
plt.title("Gravitational Wave Data")
plt.legend()
plt.show()


In [ ]:
# Bandpass filter (20–300 Hz)
def butter_bandpass_filter(data, lowcut=20, highcut=300, fs=4096, order=4):
    nyquist = 0.5 * fs
    low = lowcut / nyquist
    high = highcut / nyquist
    b, a = butter(order, [low, high], btype="band")
    return filtfilt(b, a, data)


# Apply filter
filtered_data = butter_bandpass_filter(data_cut, fs=fs, lowcut=20, highcut=300)


# Plot filtered data
plt.figure(figsize=(10, 4))
plt.plot(t_cut, filtered_data, label="Filtered Data", color="red")
plt.xlabel("Time (s)")
plt.ylabel("Strain")
plt.title("Filtered Gravitational Wave Data")
plt.legend()
plt.show()

In [ ]:
# Compute FFT of the cleaned signal
N = len(filtered_data)
fft_vals = np.fft.rfft(filtered_data)  # Compute real FFT
freqs = np.fft.rfftfreq(N, 1/fs)  # Frequency axis

# Plot power spectrum
plt.figure(figsize=(10, 4))
plt.plot(freqs, np.abs(fft_vals), color="blue", label="Fourier Transform")
plt.xlim(0, 500)  # Focus on relevant GW frequency range
plt.xlabel("Frequency (Hz)")
plt.ylabel("Magnitude")
# plt.ylim([0, np.max(np.abs(fft_vals))])
plt.yscale('log')
plt.title("Fourier Transform of Cleaned GW150914 Signal")
plt.legend()
plt.show()

In [ ]:
# Compute the spectrogram for 2D color plot
offset_filtered_data = filtered_data - np.mean(filtered_data)

nperseg_spec = fs//4  # Window size for spectrogram
f_spec, t_spec, Sxx = spectrogram(offset_filtered_data, fs, nperseg=nperseg_spec, noverlap=fs//8)

Sxx_normalized = Sxx/np.max(Sxx)

# Edit y min/max to zoom
y_plot_min = 20
y_plot_max = 300

mask = (f_spec > y_plot_min) & (f_spec < y_plot_max)  # Create a boolean mask
Sxx_filtered = Sxx_normalized[mask, :]  # Apply mask to the correct axis
f_spec_filtered = f_spec[mask]  # Keep frequency axis consistent

Sxx_filtered = Sxx_filtered/np.max(Sxx_filtered)

# Adjust color limits for better contrast
vmin, vmax = np.percentile(Sxx_filtered, [0, 100]) # percentile ranges


# Plot the 2D colorized PSD (Spectrogram)
plt.pcolormesh(t_spec, f_spec_filtered, Sxx_filtered, shading="auto", cmap="inferno", vmin=vmin, vmax=vmax)
plt.colorbar(label="Normalized Power Spectral Density")
plt.xlabel("Time (s)")
plt.ylabel("Frequency (Hz)")
plt.yscale("log")
plt.ylim(y_plot_min, y_plot_max)  # Focus on relevant GW range
plt.title("Title")
plt.show()